In [ ]:
# 把 三国演义.txt 解析 转为固定格式

In [ ]:

import json

file = "三国演义.txt"

with open(file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

res = []
r = []
i = 0
while i < len(lines)-1:

    if lines[i] == '\n' and lines[i+1] == '\n':

        res.append(r)
        r = []
        i += 2
    else:
        r.append(lines[i])
        i += 1

if len(r) > 0:
    res.append(r)
    
res1 = []
res2 = []
text_len1 = []
for x in res:
    x1 = []
    for c in x:
        c = c.replace('\n', '')
        if len(c) == 0:
            continue
        x1.append(c)
    if len(x1) > 0:
        res1.append('\n'.join(x1))
        res2.append(x1)
        text_len1 += [len(c1) for c1 in x1]

text_len = []
for x in res1:
    text_len.append(len(x))

otf = "o_data.json"
f = open(otf, 'w', encoding='utf-8')
json.dump(res2, f, ensure_ascii=False)
f.close()

# 事件大纲



In [ ]:
# 调用AI大模型（chatgpt）进行事件抽取

In [ ]:

import requests
import openai

import os
import re
import json
import copy


def get_post_openai(message):
    openai.api_key = ""

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": message}
        ]
    )

    print(completion.choices[0].message)
    return completion.choices[0].message


def get_words(text):
    prompt = """
你是一个小说家，熟读三国演义，知道三国演义里面任何一个故事情节，任何一个人物，任何一个事件，尤其是对诸葛亮（孔明）非常熟悉。下面给你一段文本，你需要从文本中进行事件抽取，抽取出事件发生的时间、地点、人物，给出事件大纲；并且判断出该事件是否涉及到诸葛亮，如果涉及请说明，诸葛亮在该事件的位置和所起的作用。如果不涉及事件，请回答“无”。一定要按照格式回答，用中文简要作答。

输入文本：
{}

以下是需要的输出:
输出事件：
时间：
<>

地点:
<>

人物:
<>

事件大纲:
<>

是否涉及诸葛亮:
<是|否>

诸葛亮在该事件中的位置:
<>

所起作用:
<>


注意：
一定要按照格式回答，用中文回答。

    """
    query = prompt.format(text)
    res = get_post_openai(query)
    return res


file = "o_data.json"
data = json.load(open(file, 'r'))


def get(data, i):
    o_path = f'./o_data/{i}'
    if not os.path.exists(o_path):
        os.makedirs(o_path, exist_ok=True)

    for j, t in enumerate(data):
        try:
            r = get_words(t)
            t1 = {"text": t, "words": r}
            o_file = os.path.join(o_path, f"{j}.json")
            f = open(o_file, 'w', encoding="utf-8")
            json.dump(t1, f, ensure_ascii=False)
            f.close()
        except:
            print(f"error  ... {i} {j}.json ...")


if __name__ == '__main__':
    for i, d in enumerate(data):
        get(d, i)

    # nohup python3 -u t1.py > log_train_0312_4.log 2>&1 &
            

In [ ]:
# 事件抽取的结果进行解析

In [ ]:

import json
import os
import re


file = "o_data.json"
o_data = json.load(open(file, 'r',encoding='utf-8'))


res = []
for i in range(120):
    a = f"./o_data/{i}/"
    r = []
    for c in os.listdir(a):
        if c.endswith('json'):
            c1 = os.path.join(a, c)
            r.append([c1, int(c[:-5])])
    r1 = sorted(r, key=lambda x: x[1])
    res.append(r1)

# i = 0
# for  a, b in zip(o_data, res):
#     if len(a) != len(b):
#         i += 1
#         print(i-1)


def get_content_between_a_b(a, b, text):
    return re.search(f"{a}(.*?)\n{b}", text, re.DOTALL).group(1).strip().replace('\n', "")


def read(file):
    
    data = json.load(open(file, 'r', encoding='utf-8'))
    
    res = {
        "时间": None,
        "地点": None,
        "人物": None,
        "事件大纲": None,
        "是否涉及诸葛亮": None,
        "诸葛亮在该事件中的位置": None,
        "所起作用": None
    }
    r = data['words']
    try:
        res["时间"] = get_content_between_a_b("时间", "地点", r)
        res["地点"] = get_content_between_a_b("地点", "人物", r)
        res["人物"] = get_content_between_a_b("人物", "事件大纲", r)
        res["事件大纲"] = get_content_between_a_b("事件大纲", "是否涉及诸葛亮", r)
        res["是否涉及诸葛亮"] = get_content_between_a_b("是否涉及诸葛亮", "诸葛亮在该事件中的位置", r)
        res["诸葛亮在该事件中的位置"] = get_content_between_a_b("诸葛亮在该事件中的位置", "所起作用", r)
        if "所起作用" in r:
            r1 = r[r.index("所起作用")+len("所起作用")+1:].replace('\n', "")
            res["所起作用"] = r1
    
    except:
        res['text'] = data['text']
        res['words'] = data['words']
        return res

    # r = data['words'].split("\n\n")
    # for a in r:
    #     if a.startswith("输出事件："):
    #         a = a[len("输出事件："):]
    #     if a.startswith("\n"):
    #         a = a[1:]
    #         
    #     a = a.split('\n')
    # 
    #     if len(a) == 2:
    #         for k, v in res.items():
    #             if a[0].startswith(k):
    #                 res[k] = a[1]
    #     elif len(a) == 3:
    #         for k, v in res.items():
    #             if a[1].startswith(k):
    #                 res[k] = a[2]
    #                 
    #     elif len(a) == 1:
    #         a = a[0]
    #         for k, v in res.items():
    #             if a.startswith(k):
    #                 res[k] = a[len(k)+1:]
    #     else:
    #         print(1)
    
    res['text'] = data['text']
    res['words'] = data['words']
    
    return res
    

result = []
for r in res:
    a = []
    for r1 in r:
        a.append(read(r1[0]))
        
    result.append(a)
    
otf = "o_res.json"
f = open(otf, 'w', encoding='utf-8')
json.dump(result, f, ensure_ascii=False)
f.close()
    
print(1)


In [ ]:
# 对结果进行分层排序 （人物， 事件大纲，原文本）

In [2]:
import json


class ZhuGeLiangImp:
    def __init__(self):
        data = json.load(open("o_res.json", 'r', encoding='utf-8'))
        self.name = "诸葛亮"
        self.name1 = "孔明"
        start_time = [0, 0]
        for i, d in enumerate(data):
            flag = False
            for j, d1 in enumerate(d):
                if "孔明" in d1['text']:
                    start_time = [i, j]
                    flag = True
                    break
            if flag:
                break
        
        self.data = data
        self.start_time = start_time
        
    def calculate_score(self):
        res_score = []
        res_text = []
        for i, d in enumerate(self.data):
            if i < self.start_time[0]:
                for j, d1 in enumerate(d):
                    res_score.append([d1, i, j, 0])
                    res_text.append(d1)
            else:
                for j, d1 in enumerate(d):
                    res_text.append(d1)
                    if j < self.start_time[1]:
                        res_score.append([d1, i, j, 0])

                    else:
                        s = 0
                        if self.name in d1['人物'] or self.name1 in d1['人物']:
                            s += 10
                            
                        t = d1['text']
                        t1 = d1['事件大纲']

                        s += len(t.split(self.name))-1
                        s += len(t.split(self.name1)) - 1
                        s += len(t1.split(self.name)) - 1
                        s += len(t1.split(self.name1)) - 1
                        res_score.append([d1, i, j, s])

        res_score1 = sorted(res_score, key=lambda x: x[-1], reverse=True)
        res_score2 =[[x[1], x[3]] for x in res_score1 if len(x) > 3]

        print(res_score2)   
        #输出章节和评分
        #print(res_score1[:3])
        print(1)


if __name__ == '__main__':
    zgl = ZhuGeLiangImp()
    zgl.calculate_score()


[[42, 50], [101, 29], [91, 28], [92, 27], [38, 26], [86, 26], [86, 26], [101, 26], [43, 24], [96, 24], [41, 23], [44, 23], [100, 23], [38, 22], [43, 22], [62, 22], [88, 22], [88, 22], [43, 21], [48, 21], [79, 21], [84, 21], [99, 21], [45, 20], [88, 20], [89, 20], [93, 20], [94, 20], [98, 20], [100, 20], [102, 20], [41, 19], [69, 19], [84, 19], [99, 19], [102, 19], [49, 18], [76, 18], [85, 18], [89, 18], [92, 18], [94, 18], [96, 18], [96, 18], [98, 18], [101, 18], [102, 18], [35, 17], [55, 17], [56, 17], [64, 17], [84, 17], [87, 17], [92, 17], [94, 17], [98, 17], [98, 17], [102, 17], [116, 17], [35, 16], [38, 16], [41, 16], [45, 16], [48, 16], [48, 16], [52, 16], [53, 16], [59, 16], [62, 16], [76, 16], [79, 16], [87, 16], [90, 16], [97, 16], [98, 16], [99, 16], [102, 16], [39, 15], [42, 15], [51, 15], [51, 15], [53, 15], [54, 15], [55, 15], [55, 15], [64, 15], [66, 15], [86, 15], [87, 15], [88, 15], [89, 15], [89, 15], [90, 15], [91, 15], [97, 15], [97, 15], [97, 15], [99, 15], [99, 15]